<a href="https://colab.research.google.com/github/Moinkhan99ds/Duplicate-Voter-detection-system/blob/main/Face_detection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile, io, os

uploaded = files.upload()
zip_name = next(iter(uploaded))

os.makedirs("db_faces", exist_ok=True)
with zipfile.ZipFile(io.BytesIO(uploaded[zip_name])) as z:
    z.extractall("db_faces")

print("✅ Database images extracted")


Saving my images.zip to my images (1).zip
✅ Database images extracted


In [ ]:
import tensorflow as tf
import numpy as np
import cv2, os
from numpy.linalg import norm

model = tf.keras.applications.MobileNetV2(include_top=False, pooling='avg')
IMG_SIZE = 224

def get_embedding(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return model.predict(img, verbose=0)[0]

def cosine_sim(a, b):
    return np.dot(a,b) / (norm(a)*norm(b))


/tmp/ipython-input-1356761353.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = tf.keras.applications.MobileNetV2(include_top=False, pooling='avg')


In [ ]:
db_path = "db_faces"
db_files = [f for f in os.listdir(db_path) if f.lower().endswith(('jpg','png','jpeg'))]
db_files.sort()

db_embeddings = {}

for f in db_files:
    img = cv2.imread(f"{db_path}/{f}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    db_embeddings[f] = get_embedding(img)

print(" Database embeddings created")

 Database embeddings created


In [ ]:
def check_new_voter(upload_path, threshold=0.75):
    img = cv2.imread(upload_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    new_emb = get_embedding(img)

    for db_name, db_emb in db_embeddings.items():
        sim = cosine_sim(new_emb, db_emb)
        if sim > threshold:
            return f"🚨 Duplicate voter found: {db_name} | Similarity: {sim:.2f}"

    new_name = f"new_{len(db_embeddings)}.jpg"
    cv2.imwrite(f"db_faces/{new_name}", cv2.imread(upload_path))
    db_embeddings[new_name] = new_emb

    return f"✅ New voter added | Stored as: {new_name}"

In [ ]:
from google.colab import files

uploaded = files.upload()
new_file = next(iter(uploaded))

result = check_new_voter(new_file)
print(result)




Saving ChatGPT Image Nov 1, 2025, 12_30_36 PM.png to ChatGPT Image Nov 1, 2025, 12_30_36 PM (4).png
🚨 Duplicate voter found: new_15.jpg | Similarity: 1.00
